In [1]:
# import kfp
# from kfp.dsl import pipeline, component, Output, Dataset, Input
# from minio import Minio
# from typing import NamedTuple
# from datetime import datetime

# # Step 1: Fetch data from Trino and save it to MinIO
# @component(base_image='bitnami/spark:3.5', packages_to_install=['trino', 'pandas', 'pyarrow', 's3fs','scikit-learn','numpy','matplotlib','seaborn','pyspark','boto3'])
# def fetch_data_trino(minio_bucket: str) -> str:
#     from trino.dbapi import connect
#     import csv
#     import boto3

#     # Trino Connection Details
#     TRINO_HOST = "192.168.80.155"
#     TRINO_PORT = "30071"
#     TRINO_USER = "ctzn.bank"
#     TRINO_PASSWORD = "ctzn.bank_123"
#     TRINO_CATALOG = "iceberg"
#     TRINO_SCHEMA = "silver_crmuser"
    
#     # Output CSV filename
#     OUTPUT_FILE = "/tmp/ctzn_10_years_data.csv"

#     # SQL Query for customer segmentation data
#     SQL_QUERY = """ 
#     WITH recent_customers AS (
#         SELECT DISTINCT 
#             g.cif_id
#         FROM 
#             gold.dim_gam AS g
#         WHERE 
#             CAST(DATE_PARSE(SUBSTRING(g.acct_opn_date, 1, 19), '%Y-%m-%dT%H:%i:%s') AS DATE) >= CURRENT_DATE - INTERVAL '5' YEAR
#     ),
    
#     account_activity AS (
#         SELECT 
#             a.cif_id,
#             -- Balance calculations
#             SUM(COALESCE(a.total_credit_tran_vol, 0) - COALESCE(a.total_debit_tran_vol, 0)) AS balance,
#             COUNT(DISTINCT a.nepali_month)/6.0 AS balance_frequency,
            
#             -- Purchase metrics
#             SUM(COALESCE(a.total_debit_tran_vol, 0)) AS purchases,
#             MAX(COALESCE(a.total_debit_tran_vol, 0)) AS oneoff_purchases,
#             SUM(COALESCE(a.total_debit_tran_vol, 0)) - MAX(COALESCE(a.total_debit_tran_vol, 0)) AS installments_purchases,
            
#             -- Cash advance calculations
#             SUM(CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 AND COALESCE(a.total_debit_tran_vol, 0) = 0 
#                      THEN COALESCE(a.total_credit_tran_vol, 0) ELSE 0 END) AS cash_advance,
            
#             -- Frequency calculations
#             COUNT(DISTINCT a.foracid)/6.0 AS purchases_frequency,
#             COUNT(DISTINCT CASE WHEN COALESCE(a.total_debit_tran_vol, 0) > 100000 THEN a.foracid END)/6.0 AS oneoff_purchases_frequency,
#             COUNT(DISTINCT CASE WHEN COALESCE(a.total_debit_tran_vol, 0) BETWEEN 1 AND 100000 THEN a.foracid END)/6.0 AS purchases_installments_frequency,
#             COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 THEN a.foracid END)/6.0 AS cash_advance_frequency,
            
#             -- Transaction counts
#             COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 THEN a.foracid END) AS cash_advance_trx,
#             COUNT(DISTINCT a.foracid) AS purchases_trx,
            
#             -- Payment behavior
#             SUM(COALESCE(a.total_credit_tran_vol, 0)) AS payments,
#             COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) >= COALESCE(a.total_debit_tran_vol, 0) 
#                                 THEN a.nepali_month END)/6.0 AS prc_full_payment
#         FROM 
#             gold.mv_fact_deposit_account_insights a
#         JOIN 
#             recent_customers rc ON a.cif_id = rc.cif_id
#         GROUP BY 
#             a.cif_id
#     ),
    
#     salary_stats AS (
#         SELECT 
#             APPROX_PERCENTILE(COALESCE(salary_per_month, 0), 0.5) AS median_salary,
#             APPROX_PERCENTILE(COALESCE(salary_per_month, 0), 0.05) AS fifth_percentile_salary
#         FROM 
#             gold.dim_customers
#     ),
    
#     customer_profile AS (
#         SELECT 
#             g.cif_id,
#             -- Tenure calculation from account opening date
#             DATE_DIFF('year', 
#                      CAST(DATE_PARSE(SUBSTRING(MIN(g.acct_opn_date), 1, 19), '%Y-%m-%dT%H:%i:%s') AS DATE), 
#                      CURRENT_DATE) AS tenure,
            
#             -- Estimated minimum payments (5% of median salary)
#             (SELECT fifth_percentile_salary FROM salary_stats) AS minimum_payments
#         FROM 
#             gold.dim_gam g
#         LEFT JOIN 
#             gold.dim_customers c ON g.cif_id = c.cif_id
#         GROUP BY 
#             g.cif_id
#     )
    
#     SELECT 
#         aa.cif_id AS custid,
#         aa.balance,
#         aa.balance_frequency,
#         aa.purchases,
#         aa.oneoff_purchases,
#         aa.installments_purchases,
#         aa.cash_advance,
#         aa.purchases_frequency,
#         aa.oneoff_purchases_frequency,
#         aa.purchases_installments_frequency,
#         aa.cash_advance_frequency,
#         aa.cash_advance_trx,
#         aa.purchases_trx,
#         -- Using estimated credit limit (3x median salary)
#         (SELECT median_salary * 3 FROM salary_stats) AS credit_limit,
#         aa.payments,
#         cp.minimum_payments,
#         aa.prc_full_payment,
#         cp.tenure
#     FROM 
#         account_activity aa
#     JOIN 
#         customer_profile cp ON aa.cif_id = cp.cif_id
#     ORDER BY 
#         aa.cif_id;
#     """

#     try:
#         # Connect to Trino
#         conn = connect(
#             host=TRINO_HOST,
#             port=TRINO_PORT,
#             user=TRINO_USER,
#             password=TRINO_PASSWORD,
#             catalog=TRINO_CATALOG,
#             schema=TRINO_SCHEMA,
#         )
#         cursor = conn.cursor()
        
#         # Execute the query
#         cursor.execute(SQL_QUERY)
        
#         # Save the result to a CSV file
#         columns = [desc[0] for desc in cursor.description]
#         with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8') as f:
#             writer = csv.writer(f)
#             writer.writerow(columns)  # Write header
#             rows = cursor.fetchall()
#             writer.writerows(rows)  # Write data

#         conn.close()
        
#         # Upload CSV to MinIO
#         minio_client = boto3.client(
#             's3',
#             endpoint_url="http://192.168.80.155:32000",
#             aws_access_key_id="admin",
#             aws_secret_access_key="dlyticaD123"
#         )
#         current_date = datetime.now().strftime("%Y-%m-%d")
#         saving_file_name = f"data/customer_segmentation_raw_data_{current_date}.csv"
#         minio_client.upload_file(OUTPUT_FILE, minio_bucket, saving_file_name)
        
#         return saving_file_name
#     except Exception as e:
#         print(f"Error occurred: {str(e)}")
#         return None

# # Step 2: Perform Feature Engineering and Customer Segmentation
# # Step 2: Perform Feature Engineering and Customer Segmentation
# @component(base_image='quay.io/datanature_dev/jupyternotebook:java_home14', packages_to_install=['trino', 'pandas', 'pyarrow', 's3fs','scikit-learn','numpy','matplotlib','seaborn','pyspark','boto3'])
# def feature_engineering_and_segmentation(file_path: str, minio_bucket: str) -> str:
#     # Initialize Spark session
#     spark = SparkSession.builder \
#         .appName("CustomerSegmentation") \
#         .config("spark.driver.memory", "30g") \
#         .config("spark.executor.memory", "30g") \
#         .config("spark.executor.memoryOverhead", "18g") \
#         .config("spark.driver.maxResultSize", "18g") \
#         .config("spark.executor.cores", "4") \
#         .config("spark.executor.instances", "4") \
#         .config("spark.dynamicAllocation.enabled", "true") \
#         .config("spark.dynamicAllocation.minExecutors", "2") \
#         .config("spark.dynamicAllocation.maxExecutors", "10") \
#         .config("spark.dynamicAllocation.initialExecutors", "4") \
#         .config("spark.default.parallelism", "200") \
#         .config("spark.sql.shuffle.partitions", "200") \
#         .enableHiveSupport() \
#         .getOrCreate()

#     # Initialize MinIO client
#     minio_client = boto3.client(
#         's3',
#         endpoint_url="http://192.168.80.155:32000",
#         aws_access_key_id="admin",
#         aws_secret_access_key="dlyticaD123"
#     )
    
#     # Download raw data from MinIO
#     local_raw_data_path = "/tmp/raw_data.csv"
#     minio_client.download_file(minio_bucket, file_path, local_raw_data_path)
    
#     # Load Data
#     df = spark.read.csv(local_raw_data_path, header=True, inferSchema=True)

#     # Display basic information
#     print("Total number of records:", df.count())
#     print("Schema:")
#     df.printSchema()

#     # Handle missing values (fill with mean for numerical columns, "Unknown" for categorical columns)
#     def handle_missing_values(df):
#         categorical_cols = ['gender', 'employment_status', 'marital_status', 'occupation', 'age_group', 'withdrawal_trends']
#         numerical_cols = [
#             'total_accounts', 'total_saving_accounts', 'total_fixed_accounts', 'total_loan_accounts',
#             'total_overdraft_accounts', 'total_debit_transaction_count', 'total_credit_transaction_count',
#             'total_debit_transaction_amount', 'total_credit_transaction_amount', 'average_debit_transaction_count',
#             'average_credit_transaction_count', 'average_debit_amount', 'average_credit_amount',
#             'average_yearly_saving', 'total_yearly_saving', 'total_dormant_days', 'total_dormant_account',
#             'total_active_account', 'first_account_opened_days', 'last_account_opened_days'
#         ]
        
#         # Fill missing values
#         df = df.fillna(0, subset=numerical_cols)
#         df = df.fillna("Unknown", subset=categorical_cols)
        
#         return df

#     df = handle_missing_values(df)

#     # Handle outliers using Z-score method (already implemented in your code)
#     def handle_outliers_zscore(df, cols, threshold=3.0):
#         """Handle outliers in spark dataframe using z-score method"""
#         for c in cols:
#             stats = df.select(mean(col(c)).alias("mean"), stddev(col(c)).alias("stddev")).collect()[0]
#             mean_val, stddev_val = stats["mean"], stats["stddev"]
#             if stddev_val is None or stddev_val == 0:
#                 continue
#             df = df.withColumn(
#                 c,
#                 when((col(c) - mean_val) / stddev_val > threshold, mean_val + threshold * stddev_val)
#                 .when((col(c) - mean_val) / stddev_val < -threshold, mean_val - threshold * stddev_val)
#                 .otherwise(col(c))
#             )
#         return df

#     numeric_cols = [field.name for field in df.schema.fields 
#                     if isinstance(df.schema[field.name].dataType, DoubleType) 
#                     or str(df.schema[field.name].dataType).startswith('IntegerType') 
#                     or str(df.schema[field.name].dataType).startswith('LongType')]
#     df = handle_outliers_zscore(df, numeric_cols)

#     # Create feature engineering columns (as per your provided code)
#     df = df.withColumn("new_balance_balance_frequency", col("balance") * col("balance_frequency"))
#     df = df.withColumn("new_oneoff_purchases_purchases", 
#                       when(col("purchases") != 0, col("oneoff_purchases") / col("purchases")).otherwise(0))
#     df = df.withColumn("new_installments_purchases_purchases", 
#                       when(col("purchases") != 0, col("installments_purchases") / col("purchases")).otherwise(0))
#     df = df.withColumn("new_cash_advance_purchases_purchases", col("cash_advance") * col("cash_advance_frequency"))
#     df = df.withColumn("new_purchases_purchases_frequency", col("purchases") * col("purchases_frequency"))
#     df = df.withColumn("new_purchases_oneoff_purchases_frequency", col("purchases") * col("oneoff_purchases_frequency"))
#     df = df.withColumn("new_purchases_purchases_trx", 
#                       when(col("purchases_trx") != 0, col("purchases") / col("purchases_trx")).otherwise(0))
#     df = df.withColumn("new_cash_advance_cash_advance_trx", 
#                       when(col("cash_advance_trx") != 0, col("cash_advance") / col("cash_advance_trx")).otherwise(0))
#     df = df.withColumn("new_balance_credit_limit", 
#                       when(col("credit_limit") != 0, col("balance") / col("credit_limit")).otherwise(0))
#     df = df.withColumn("new_payments_credit_limit", 
#                       when(col("minimum_payments") != 0, col("payments") / col("minimum_payments")).otherwise(0))

#     # Fill remaining NaN values with 0
#     df = df.na.fill(0)

#     # Prepare features for clustering (drop cif_id)
#     if "cif_id" in df.columns:
#         features_df = df.drop("cif_id")
#     else:
#         features_df = df

#     # Filter out non-numeric columns for the assembler
#     numeric_cols = [col_name for col_name, dtype in features_df.dtypes 
#                     if dtype != 'string' and not dtype.startswith('array') 
#                     and not col_name == 'cif_id']

#     # Assemble features into a vector
#     assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")
#     assembled_data = assembler.transform(features_df)

#     # Standardize features
#     scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
#     scaler_model = scaler.fit(assembled_data)
#     scaled_data = scaler_model.transform(assembled_data)

#     # Apply PCA for dimensionality reduction
#     pca = PCA(k=2, inputCol="scaled_features", outputCol="pca_features")
#     pca_model = pca.fit(scaled_data)
#     pca_data = pca_model.transform(scaled_data)

#     # Explained variance ratio
#     explained_variance = pca_model.explainedVariance.toArray()
#     print("Explained variance ratio:", explained_variance)
#     print("Total variance explained:", sum(explained_variance))

#     # Find optimal number of clusters using Silhouette score and Elbow method
#     def find_optimal_clusters(data, max_k=10):
#         costs = []
#         silhouettes = []
#         evaluator = ClusteringEvaluator(predictionCol="prediction", featuresCol="pca_features", 
#                                        metricName="silhouette")
        
#         for k in range(2, max_k + 1):
#             kmeans = KMeans(k=k, seed=42, featuresCol="pca_features")
#             model = kmeans.fit(data)
#             cost = model.summary.trainingCost
#             costs.append(cost)
            
#             # Make predictions
#             predictions = model.transform(data)
#             silhouette = evaluator.evaluate(predictions)
#             silhouettes.append(silhouette)
            
#             print(f"k={k}, Cost={cost}, Silhouette Score={silhouette}")
        
#         return costs, silhouettes

#     costs, silhouettes = find_optimal_clusters(pca_data, max_k=10)

#     # Determine optimal k based on silhouette score
#     optimal_k = silhouette_df.loc[silhouette_df['silhouette'].idxmax(), 'k']
#     print(f"Optimal number of clusters based on silhouette score: {optimal_k}")

#     kmeans = KMeans(k=optimal_k, seed=42, featuresCol="pca_features")
#     kmeans_model = kmeans.fit(pca_data)
#     clustered_data = kmeans_model.transform(pca_data)

#     # Prepare results for cluster visualization and analysis
#     pandas_df = clustered_data.select("pca_features", "prediction").toPandas()
#     pandas_df['pca1'] = pandas_df['pca_features'].apply(lambda x: float(x[0]))
#     pandas_df['pca2'] = pandas_df['pca_features'].apply(lambda x: float(x[1]))

#     # Plot clusters
#     plt.figure(figsize=(10, 6))
#     scatter = plt.scatter(pandas_df['pca1'], pandas_df['pca2'], c=pandas_df['prediction'], cmap='viridis', alpha=0.5)
#     plt.colorbar(scatter, label='Cluster')
#     plt.title('Customer Segmentation - PCA Visualization')
#     plt.xlabel('Principal Component 1')
#     plt.ylabel('Principal Component 2')
#     cluster_image_path = "/tmp/cluster_visualization.png"
#     plt.savefig(cluster_image_path)

#     # Save visualization to MinIO
#     # Save visualization to MinIO
#     minio_client.upload_file(cluster_image_path, minio_bucket, f"images/customer_segmentation_{datetime.now().strftime('%Y-%m-%d')}_cluster_visualization.png")

#     # Save clustering results to CSV
#     final_output_path = f"/tmp/customer_segmentation_output_{datetime.now().strftime('%Y-%m-%d')}.csv"
#     clustered_data.toPandas().to_csv(final_output_path, index=False)

#     # Upload the output file to MinIO
#     minio_client.upload_file(final_output_path, minio_bucket, f"data/customer_segmentation_output_{datetime.now().strftime('%Y-%m-%d')}.csv")

#     return final_output_path

# # Step 3: Save final cluster predictions into Trino
# @component(base_image='bitnami/spark:3.5', packages_to_install=['trino','minio','boto3'])
# def save_predictions_to_trino(prediction_file: str) -> None:
#     from trino.dbapi import connect
#     import pandas as pd

#     # Trino Connection Details
#     TRINO_HOST = "192.168.80.155"
#     TRINO_PORT = "30071"
#     TRINO_USER = "ctzn.bank"
#     TRINO_PASSWORD = "ctzn.bank_123"
#     TRINO_CATALOG = "iceberg"
#     TRINO_SCHEMA = "silver_crmuser"

#     # Load the predictions data from MinIO (already saved in previous step)
#     predictions_df = pd.read_csv(prediction_file)

#     # Save predictions to Trino (Iceberg catalog)
#     conn = connect(
#         host=TRINO_HOST,
#         port=TRINO_PORT,
#         user=TRINO_USER,
#         password=TRINO_PASSWORD,
#         catalog=TRINO_CATALOG,
#         schema=TRINO_SCHEMA,
#     )
#     cursor = conn.cursor()

#     # Assume predictions_df contains 'cif_id' and 'prediction' columns
#     for index, row in predictions_df.iterrows():
#         cif_id = row['cif_id']
#         prediction = row['prediction']
        
#         # Insert prediction data into Trino
#         cursor.execute(
#             f"INSERT INTO {TRINO_CATALOG}.{TRINO_SCHEMA}.customer_predictions (cif_id, prediction) VALUES ({cif_id}, {prediction})"
#         )

#     conn.commit()
#     conn.close()

# # Step 4: Define the full pipeline
# @pipeline(name="Customer Segmentation Pipeline")
# def customer_segmentation_pipeline(minio_bucket: str = "ai360ctzn-customer-segmentation"):
#     # Step 1: Fetch Data from Trino and save it to MinIO
#     data_task = fetch_data_trino(minio_bucket=minio_bucket)

#     # Step 2: Perform Feature Engineering and Customer Segmentation
#     segmentation_task = feature_engineering_and_segmentation(file_path=data_task.output, minio_bucket=minio_bucket)

#     # Step 3: Save predictions into Trino
#     save_predictions_task = save_predictions_to_trino(prediction_file=segmentation_task.output)

# # Compile the pipeline
# from kfp import compiler
# compiler.Compiler().compile(customer_segmentation_pipeline, "customer_segmentation_pipeline.yaml")

# print("Pipeline compiled successfully.")


Pipeline compiled successfully.


In [3]:
pip install kfp minio trino pandas numpy matlpotlib

  Using cached minio-7.2.15-py3-none-any.whl.metadata (6.7 kB)
  Using cached trino-0.333.0-py3-none-any.whl.metadata (19 kB)
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement matlpotlib (from versions: none)
ERROR: No matching distribution found for matlpotlib
Note: you may need to restart the kernel to use updated packages.


### Final corrected pipeline

In [76]:
import kfp
from kfp.dsl import pipeline, component
from datetime import datetime
from typing import NamedTuple

# Step 1: Fetch data from Trino and save it to MinIO
@component(
    base_image='bitnami/spark:3.5', 
    packages_to_install=['trino', 'pandas', 'pyarrow', 's3fs', 'boto3', 'urllib3']
)
def fetch_data_trino(minio_bucket: str) -> str:
    import trino
    from trino.auth import BasicAuthentication
    import csv
    import boto3
    import os
    import urllib3
    from datetime import datetime
    
    # Disable SSL warnings
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

    # Trino Connection Details
    TRINO_HOST = "192.168.80.155"
    TRINO_PORT = "30071"
    TRINO_USER = "ctzn.bank"
    TRINO_PASSWORD = "ctzn.bank_123"
    TRINO_CATALOG = "iceberg"
    TRINO_SCHEMA = "silver_crmuser"
    TRINO_HTTP_SCHEME = "https"

    # Output file
    OUTPUT_FILE = "/tmp/ctzn_10_years_data.csv"

    # SQL Query
    SQL_QUERY = """
    WITH recent_customers AS (
        SELECT DISTINCT g.cif_id
        FROM gold.dim_gam AS g
        WHERE CAST(DATE_PARSE(SUBSTRING(g.acct_opn_date, 1, 19), '%Y-%m-%dT%H:%i:%s') AS DATE) >= CURRENT_DATE - INTERVAL '10' YEAR
    ),
    account_activity AS (
        SELECT 
            a.cif_id,
            SUM(COALESCE(a.total_credit_tran_vol, 0) - COALESCE(a.total_debit_tran_vol, 0)) AS balance,
            COUNT(DISTINCT a.nepali_month)/6.0 AS balance_frequency,
            SUM(COALESCE(a.total_debit_tran_vol, 0)) AS purchases,
            MAX(COALESCE(a.total_debit_tran_vol, 0)) AS oneoff_purchases,
            SUM(COALESCE(a.total_debit_tran_vol, 0)) - MAX(COALESCE(a.total_debit_tran_vol, 0)) AS installments_purchases,
            SUM(CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 AND COALESCE(a.total_debit_tran_vol, 0) = 0 
                    THEN COALESCE(a.total_credit_tran_vol, 0) ELSE 0 END) AS cash_advance,
            COUNT(DISTINCT a.foracid)/6.0 AS purchases_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_debit_tran_vol, 0) > 100000 THEN a.foracid END)/6.0 AS oneoff_purchases_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_debit_tran_vol, 0) BETWEEN 1 AND 100000 THEN a.foracid END)/6.0 AS purchases_installments_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 THEN a.foracid END)/6.0 AS cash_advance_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 THEN a.foracid END) AS cash_advance_trx,
            COUNT(DISTINCT a.foracid) AS purchases_trx,
            SUM(COALESCE(a.total_credit_tran_vol, 0)) AS payments,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) >= COALESCE(a.total_debit_tran_vol, 0) 
                                THEN a.nepali_month END)/6.0 AS prc_full_payment
        FROM gold.mv_fact_deposit_account_insights a
        JOIN recent_customers rc ON a.cif_id = rc.cif_id
        GROUP BY a.cif_id
    ),
    salary_stats AS (
        SELECT 
            APPROX_PERCENTILE(COALESCE(salary_per_month, 0), 0.5) AS median_salary,
            APPROX_PERCENTILE(COALESCE(salary_per_month, 0), 0.05) AS fifth_percentile_salary
        FROM gold.dim_customers
    ),
    customer_profile AS (
        SELECT 
            g.cif_id,
            DATE_DIFF('year', 
                     CAST(DATE_PARSE(SUBSTRING(MIN(g.acct_opn_date), 1, 19), '%Y-%m-%dT%H:%i:%s') AS DATE), 
                     CURRENT_DATE) AS tenure,
            (SELECT fifth_percentile_salary FROM salary_stats) AS minimum_payments
        FROM gold.dim_gam g
        LEFT JOIN gold.dim_customers c ON g.cif_id = c.cif_id
        GROUP BY g.cif_id
    )
    SELECT 
        aa.cif_id AS custid,
        aa.balance,
        aa.balance_frequency,
        aa.purchases,
        aa.oneoff_purchases,
        aa.installments_purchases,
        aa.cash_advance,
        aa.purchases_frequency,
        aa.oneoff_purchases_frequency,
        aa.purchases_installments_frequency,
        aa.cash_advance_frequency,
        aa.cash_advance_trx,
        aa.purchases_trx,
        (SELECT median_salary * 3 FROM salary_stats) AS credit_limit,
        aa.payments,
        cp.minimum_payments,
        aa.prc_full_payment,
        cp.tenure
    FROM account_activity aa
    JOIN customer_profile cp ON aa.cif_id = cp.cif_id
    ORDER BY aa.cif_id
    """

    try:
        # Connect to Trino
        conn = trino.dbapi.connect(
            host=TRINO_HOST,
            port=TRINO_PORT,
            user=TRINO_USER,
            auth=BasicAuthentication(TRINO_USER, TRINO_PASSWORD),
            catalog=TRINO_CATALOG,
            schema=TRINO_SCHEMA,
            http_scheme=TRINO_HTTP_SCHEME,
            request_timeout=600,
            verify=False
        )
        cursor = conn.cursor()
        
        # Execute query and fetch data
        cursor.execute(SQL_QUERY)
        columns = [desc[0] for desc in cursor.description]
        
        # Write to CSV in batches
        with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(columns)
            
            while True:
                rows = cursor.fetchmany(1000)
                if not rows:
                    break
                writer.writerows(rows)
        
        # Upload to MinIO
        minio_client = boto3.client(
            's3',
            endpoint_url="http://192.168.80.155:32000",
            aws_access_key_id="admin",
            aws_secret_access_key="dlyticaD123",
            verify=False
        )
        
        current_date = datetime.now().strftime("%Y-%m-%d")
        minio_path = f"data/customer_segmentation_raw_{current_date}.csv"
        minio_client.upload_file(OUTPUT_FILE, minio_bucket, minio_path)
        
        return minio_path
        
    except Exception as e:
        print(f"Error in fetch_data_trino: {str(e)}")
        raise
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()


@component(
    base_image='quay.io/datanature_dev/jupyternotebook:java_home14',
    packages_to_install=[
        'pyspark', 'pandas', 'boto3', 'scikit-learn', 
        'matplotlib', 'numpy', 'trino', 'urllib3'
    ]
)
def feature_engineering_and_segmentation(
    file_path: str, 
    minio_bucket: str
) -> NamedTuple('Outputs', [
    ('segmentation_path', str),
    ('model_path', str),
    ('cluster_plot_path', str)
]):
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, when, mean, monotonically_increasing_id, stddev, count, abs, lit
    from pyspark.sql.types import DoubleType, StringType
    from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA
    from pyspark.ml.clustering import BisectingKMeans
    from pyspark.ml.evaluation import ClusteringEvaluator
    from pyspark.sql.functions import udf
    import boto3
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from datetime import datetime
    from collections import namedtuple
    import urllib3
    import os

    # Disable SSL warnings
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

    # Initialize Spark with optimized settings
    spark = SparkSession.builder \
        .appName("AdvancedCustomerSegmentation") \
        .config("spark.driver.memory", "8g") \
        .config("spark.executor.memory", "8g") \
        .config("spark.sql.shuffle.partitions", "200") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
        .enableHiveSupport() \
        .getOrCreate()

    # Initialize MinIO client
    minio_client = boto3.client(
        's3',
        endpoint_url="http://192.168.80.155:32000",
        aws_access_key_id="admin",
        aws_secret_access_key="dlyticaD123",
        verify=False
    )

    try:
        today = datetime.now().strftime('%Y-%m-%d')
        
        # Download data from MinIO
        local_path = "/tmp/raw_data.csv"
        minio_client.download_file(minio_bucket, file_path, local_path)
        df = spark.read.csv(local_path, header=True, inferSchema=True)

        if 'custid' in df.columns:
            df = df.withColumnRenamed("custid", "cif_id")

        # Enhanced feature engineering
        df = df.withColumn("balance_utilization_ratio", 
            when(col("credit_limit") != 0, col("balance") / col("credit_limit")).otherwise(0))
        df = df.withColumn("cash_advance_intensity", 
            when(col("credit_limit") != 0, col("cash_advance") / col("credit_limit")).otherwise(0))
        df = df.withColumn("payment_effort_ratio", 
            when(col("minimum_payments") != 0, col("payments") / col("minimum_payments")).otherwise(0))
        df = df.withColumn("purchase_diversity", 
            abs(col("oneoff_purchases") - col("installments_purchases")) / 
            (abs(col("oneoff_purchases") + col("installments_purchases")) + 1e-10))
        df = df.na.fill(0)

        # Feature selection
        numeric_cols = [
            'purchases', 'oneoff_purchases', 'installments_purchases', 
            'cash_advance', 'cash_advance_frequency', 'balance', 'credit_limit', 
            'payments', 'minimum_payments', 'balance_utilization_ratio',
            'cash_advance_intensity', 'payment_effort_ratio', 'purchase_diversity'
        ]

        # Feature transformation pipeline
        assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")
        assembled = assembler.transform(df.drop("cif_id"))
        scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
        scaled_data = scaler.fit(assembled).transform(assembled)

        # Dimensionality reduction
        pca = PCA(k=5, inputCol="scaled_features", outputCol="pca_features")
        pca_data = pca.fit(scaled_data).transform(scaled_data)

        # Clustering optimization
        evaluator = ClusteringEvaluator(featuresCol='pca_features')
        k_range = range(6, 15)
        k_metrics = []

        for k in k_range:
            kmeans = BisectingKMeans(k=k, seed=42, featuresCol="pca_features")
            model = kmeans.fit(pca_data)
            predictions = model.transform(pca_data)
            
            cost = model.summary.trainingCost if hasattr(model.summary, 'trainingCost') else 0
            silhouette = evaluator.evaluate(predictions)
            
            centers = model.clusterCenters()
            separation = np.mean([np.min(np.linalg.norm(centers[i] - centers[j])) 
                               for i in range(len(centers)) 
                               for j in range(i+1, len(centers))])
            
            k_metrics.append({'k': k, 'cost': cost, 'silhouette': silhouette, 'separation': separation})

        k_metrics = pd.DataFrame(k_metrics)
        k_metrics['normalized_cost'] = (k_metrics['cost'] - k_metrics['cost'].min()) / (k_metrics['cost'].max() - k_metrics['cost'].min())
        k_metrics['normalized_silhouette'] = (k_metrics['silhouette'] - k_metrics['silhouette'].min()) / (k_metrics['silhouette'].max() - k_metrics['silhouette'].min())
        k_metrics['normalized_separation'] = (k_metrics['separation'] - k_metrics['separation'].min()) / (k_metrics['separation'].max() - k_metrics['separation'].min())
        k_metrics['composite_score'] = (
            0.3 * (1 - k_metrics['normalized_cost']) +
            0.4 * k_metrics['normalized_silhouette'] +
            0.3 * k_metrics['normalized_separation']
        )
        
        optimal_k = k_metrics.loc[k_metrics['composite_score'].idxmax(), 'k']

        # Plot cluster diagnostics
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15))
        ax1.plot(k_metrics['k'], k_metrics['cost'], marker='o', color='blue')
        ax1.set_xlabel('Number of Clusters (k)')
        ax1.set_ylabel('Cost', color='blue')
        ax1.grid(True)

        ax2.plot(k_metrics['k'], k_metrics['silhouette'], marker='x', color='green')
        ax2.set_xlabel('Number of Clusters (k)')
        ax2.set_ylabel('Silhouette Score', color='green')
        ax2.grid(True)

        ax3.plot(k_metrics['k'], k_metrics['separation'], marker='^', color='red')
        ax3.set_xlabel('Number of Clusters (k)')
        ax3.set_ylabel('Cluster Separation', color='red')
        ax3.grid(True)

        plt.tight_layout()
        cluster_plot_path = "/tmp/cluster_diagnostics.png"
        plt.savefig(cluster_plot_path)
        plt.close()

        # Final clustering
        kmeans = BisectingKMeans(k=optimal_k, seed=42, featuresCol="pca_features")
        model = kmeans.fit(pca_data)
        clustered_data = model.transform(pca_data)

        # Add cluster info to original data
        df_with_id = df.withColumn("row_id", monotonically_increasing_id())
        clustered_data = clustered_data.withColumn("row_id", monotonically_increasing_id())
        clustered_data = clustered_data.join(df_with_id.select("row_id", "cif_id"), on="row_id").drop("row_id")

        # Generate cluster profiles and interpretations
        summary = clustered_data.groupBy("prediction").agg(
            *[mean(col(c)).alias(c) for c in numeric_cols],
            count("cif_id").alias("cluster_size")
        )
        cluster_profiles = summary.toPandas().set_index("prediction")
        
        # Create a mapping dictionary for cluster interpretations
        interpretations = {}
        for cluster in cluster_profiles.index:
            components = []
            if 'purchases' in cluster_profiles.columns:
                spend = cluster_profiles.loc[cluster, 'purchases']
                mean_spend = cluster_profiles['purchases'].mean()
                if spend > mean_spend + 1.5 * cluster_profiles['purchases'].std():
                    components.append("High Spender")
                elif spend < mean_spend - 1.5 * cluster_profiles['purchases'].std():
                    components.append("Low Spender")
                else:
                    components.append("Moderate Spender")
            
            if 'cash_advance' in cluster_profiles.columns:
                cash = cluster_profiles.loc[cluster, 'cash_advance']
                if cash > cluster_profiles['cash_advance'].mean() * 2:
                    components.append("Heavy Cash Advance User")
            
            interpretations[cluster] = "; ".join(components) if components else "Standard Customer"

        # Create a broadcast variable for the interpretations
        interpretations_bc = spark.sparkContext.broadcast(interpretations)

        # Define a function to get interpretation that will work with Spark
        def get_interpretation(cluster_id):
            return interpretations_bc.value.get(cluster_id, "Unknown")

        # Register the UDF
        from pyspark.sql.types import StringType
        get_interpretation_udf = udf(get_interpretation, StringType())

        # Apply the interpretation
        clustered_data = clustered_data.withColumn(
            "interpretation", 
            get_interpretation_udf(col("prediction"))
        )

        # Save results to MinIO
        final_df = pd.DataFrame(
            clustered.select("cif_id", "prediction", "interpretation")
                 .rdd.map(lambda row: row.asDict())
                 .collect()
            )

        # Save CSV to MinIO
        final_csv = f"/tmp/segmentation_{today}.csv"
        final_df.to_csv(final_csv, index=False)
        minio_seg_path = f"results/segmentation_{today}.csv"
        minio_client.upload_file(final_csv, minio_bucket, minio_seg_path)
        
        # Save plot to MinIO
        minio_plot_path = f"results/cluster_diagnostics_{today}.png"
        minio_client.upload_file(cluster_plot_path, minio_bucket, minio_plot_path)
        
        # Save model to MinIO
        model_path = f"/tmp/segmentation_model_{today}"
        model.write().overwrite().save(model_path)
        minio_model_path = f"models/segmentation_model_{today}"
        for root, _, files in os.walk(model_path):
            for file in files:
                file_path = os.path.join(root, file)
                relative_path = os.path.relpath(file_path, model_path)
                minio_client.upload_file(
                    file_path, 
                    minio_bucket, 
                    f"{minio_model_path}/{relative_path}"
                )

        # Save to Trino
        from trino.dbapi import connect
        from trino.auth import BasicAuthentication
        
        conn = connect(
            host="192.168.80.155",
            port=30071,
            user="ctzn.bank",
            auth=BasicAuthentication("ctzn.bank", "ctzn.bank_123"),
            catalog="iceberg",
            schema="silver_crmuser",
            http_scheme="https",
            verify=False
        )
        cursor = conn.cursor()

        # Create table if not exists
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS iceberg.silver_crmuser.cluster_predictions (
            cif_id VARCHAR,
            cluster_id INTEGER,
            interpretation VARCHAR,
            processed_date TIMESTAMP
        ) WITH (
            partitioning = ARRAY['processed_date']
        )
        """)

        # Insert predictions in batches
        batch_size = 1000
        for i in range(0, len(final_df), batch_size):
            batch = final_df[i:i+batch_size]
            values = [
                (str(row['cif_id']), int(row['prediction']), str(row['interpretation']))
                for _, row in batch.iterrows()
            ]
            
            cursor.executemany("""
            INSERT INTO iceberg.silver_crmuser.cluster_predictions 
            (cif_id, cluster_id, interpretation, processed_date) 
            VALUES (?, ?, ?, CURRENT_TIMESTAMP)
            """, values)
            conn.commit()

        cursor.close()
        conn.close()

        output = namedtuple('Outputs', ['segmentation_path', 'model_path', 'cluster_plot_path'])
        return output(minio_seg_path, minio_model_path, minio_plot_path)
        
    except Exception as e:
        print(f"Error in feature_engineering_and_segmentation: {str(e)}")
        raise
    finally:
        spark.stop()

@component(
    base_image='bitnami/spark:3.5',
    packages_to_install=['trino', 'pandas', 'boto3', 'urllib3']
)
def save_predictions_to_trino(
    segmentation_path: str,
    minio_bucket: str
) -> str:
    import trino
    from trino.auth import BasicAuthentication
    import pandas as pd
    import boto3
    import urllib3
    
    # Disable SSL warnings
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

    try:
        # Download predictions from MinIO
        minio_client = boto3.client(
            's3',
            endpoint_url="http://192.168.80.155:32000",
            aws_access_key_id="admin",
            aws_secret_access_key="dlyticaD123",
            verify=False
        )
        
        local_path = "/tmp/predictions.csv"
        minio_client.download_file(minio_bucket, segmentation_path, local_path)
        predictions = pd.read_csv(local_path)

        # Connect to Trino
        conn = trino.dbapi.connect(
            host="192.168.80.155",
            port=30071,
            user="ctzn.bank",
            auth=BasicAuthentication("ctzn.bank", "ctzn.bank_123"),
            catalog="iceberg",
            schema="silver_crmuser",
            http_scheme="https",
            verify=False
        )
        cursor = conn.cursor()

        # Create table if not exists (redundant check for robustness)
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS iceberg.silver_crmuser.cluster_predictions (
            cif_id VARCHAR,
            cluster_id INTEGER,
            interpretation VARCHAR,
            processed_date TIMESTAMP
        ) WITH (
            partitioning = ARRAY['processed_date']
        )
        """)

        # Insert predictions in batches
        batch_size = 1000
        for i in range(0, len(predictions), batch_size):
            batch = predictions[i:i+batch_size]
            values = [
                (str(row['cif_id']), int(row['prediction']), str(row['interpretation']))
                for _, row in batch.iterrows()
            ]
            
            cursor.executemany("""
            INSERT INTO iceberg.silver_crmuser.cluster_predictions 
            (cif_id, cluster_id, interpretation, processed_date) 
            VALUES (?, ?, ?, CURRENT_TIMESTAMP)
            """, values)
            conn.commit()

        return f"Successfully saved {len(predictions)} predictions to Trino"
        
    except Exception as e:
        print(f"Error in save_predictions_to_trino: {str(e)}")
        raise
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

# Pipeline definition
@pipeline(name="Customer_Segmentation_Pipeline")
def customer_segmentation_pipeline(
    minio_bucket: str = "ai360ctzn-customer-segmentation"
):
    # Step 1: Fetch data
    fetch_task = fetch_data_trino(minio_bucket=minio_bucket)
    
    # Step 2: Process data and create segments
    segmentation_task = feature_engineering_and_segmentation(
        file_path=fetch_task.output,
        minio_bucket=minio_bucket
    )
    
    # Step 3: Save results to Trino (redundant save for robustness)
    save_task = save_predictions_to_trino(
        segmentation_path=segmentation_task.outputs['segmentation_path'],
        minio_bucket=minio_bucket
    )

# Compile the pipeline
if __name__ == "__main__":
    from kfp import compiler
    compiler.Compiler().compile(
        customer_segmentation_pipeline,
        "customer_segmentation_pipeline.yaml"
    )
    print("Pipeline compiled successfully.")

Pipeline compiled successfully.


In [77]:
import kfp
from kfp.dsl import pipeline, component
from datetime import datetime
from typing import NamedTuple

# Step 1: Fetch data from Trino and save it to MinIO
@component(
    base_image='bitnami/spark:3.5', 
    packages_to_install=['trino', 'pandas', 'pyarrow', 's3fs', 'boto3', 'urllib3']
)
def fetch_data_trino(minio_bucket: str) -> str:
    import trino
    from trino.auth import BasicAuthentication
    import csv
    import boto3
    import os
    import urllib3
    from datetime import datetime
    
    # Disable SSL warnings
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

    # Trino Connection Details
    TRINO_HOST = "192.168.80.155"
    TRINO_PORT = "30071"
    TRINO_USER = "ctzn.bank"
    TRINO_PASSWORD = "ctzn.bank_123"
    TRINO_CATALOG = "iceberg"
    TRINO_SCHEMA = "silver_crmuser"
    TRINO_HTTP_SCHEME = "https"

    # Output file
    OUTPUT_FILE = "/tmp/ctzn_10_years_data.csv"

    # SQL Query
    SQL_QUERY = """
    WITH recent_customers AS (
        SELECT DISTINCT g.cif_id
        FROM gold.dim_gam AS g
        WHERE CAST(DATE_PARSE(SUBSTRING(g.acct_opn_date, 1, 19), '%Y-%m-%dT%H:%i:%s') AS DATE) >= CURRENT_DATE - INTERVAL '10' YEAR
    ),
    account_activity AS (
        SELECT 
            a.cif_id,
            SUM(COALESCE(a.total_credit_tran_vol, 0) - COALESCE(a.total_debit_tran_vol, 0)) AS balance,
            COUNT(DISTINCT a.nepali_month)/6.0 AS balance_frequency,
            SUM(COALESCE(a.total_debit_tran_vol, 0)) AS purchases,
            MAX(COALESCE(a.total_debit_tran_vol, 0)) AS oneoff_purchases,
            SUM(COALESCE(a.total_debit_tran_vol, 0)) - MAX(COALESCE(a.total_debit_tran_vol, 0)) AS installments_purchases,
            SUM(CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 AND COALESCE(a.total_debit_tran_vol, 0) = 0 
                    THEN COALESCE(a.total_credit_tran_vol, 0) ELSE 0 END) AS cash_advance,
            COUNT(DISTINCT a.foracid)/6.0 AS purchases_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_debit_tran_vol, 0) > 100000 THEN a.foracid END)/6.0 AS oneoff_purchases_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_debit_tran_vol, 0) BETWEEN 1 AND 100000 THEN a.foracid END)/6.0 AS purchases_installments_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 THEN a.foracid END)/6.0 AS cash_advance_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 THEN a.foracid END) AS cash_advance_trx,
            COUNT(DISTINCT a.foracid) AS purchases_trx,
            SUM(COALESCE(a.total_credit_tran_vol, 0)) AS payments,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) >= COALESCE(a.total_debit_tran_vol, 0) 
                                THEN a.nepali_month END)/6.0 AS prc_full_payment
        FROM gold.mv_fact_deposit_account_insights a
        JOIN recent_customers rc ON a.cif_id = rc.cif_id
        GROUP BY a.cif_id
    ),
    salary_stats AS (
        SELECT 
            APPROX_PERCENTILE(COALESCE(salary_per_month, 0), 0.5) AS median_salary,
            APPROX_PERCENTILE(COALESCE(salary_per_month, 0), 0.05) AS fifth_percentile_salary
        FROM gold.dim_customers
    ),
    customer_profile AS (
        SELECT 
            g.cif_id,
            DATE_DIFF('year', 
                     CAST(DATE_PARSE(SUBSTRING(MIN(g.acct_opn_date), 1, 19), '%Y-%m-%dT%H:%i:%s') AS DATE), 
                     CURRENT_DATE) AS tenure,
            (SELECT fifth_percentile_salary FROM salary_stats) AS minimum_payments
        FROM gold.dim_gam g
        LEFT JOIN gold.dim_customers c ON g.cif_id = c.cif_id
        GROUP BY g.cif_id
    )
    SELECT 
        aa.cif_id AS custid,
        aa.balance,
        aa.balance_frequency,
        aa.purchases,
        aa.oneoff_purchases,
        aa.installments_purchases,
        aa.cash_advance,
        aa.purchases_frequency,
        aa.oneoff_purchases_frequency,
        aa.purchases_installments_frequency,
        aa.cash_advance_frequency,
        aa.cash_advance_trx,
        aa.purchases_trx,
        (SELECT median_salary * 3 FROM salary_stats) AS credit_limit,
        aa.payments,
        cp.minimum_payments,
        aa.prc_full_payment,
        cp.tenure
    FROM account_activity aa
    JOIN customer_profile cp ON aa.cif_id = cp.cif_id
    ORDER BY aa.cif_id
    """

    try:
        # Connect to Trino
        conn = trino.dbapi.connect(
            host=TRINO_HOST,
            port=TRINO_PORT,
            user=TRINO_USER,
            auth=BasicAuthentication(TRINO_USER, TRINO_PASSWORD),
            catalog=TRINO_CATALOG,
            schema=TRINO_SCHEMA,
            http_scheme=TRINO_HTTP_SCHEME,
            request_timeout=600,
            verify=False
        )
        cursor = conn.cursor()
        
        # Execute query and fetch data
        cursor.execute(SQL_QUERY)
        columns = [desc[0] for desc in cursor.description]
        
        # Write to CSV in batches
        with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(columns)
            
            while True:
                rows = cursor.fetchmany(1000)
                if not rows:
                    break
                writer.writerows(rows)
        
        # Upload to MinIO
        minio_client = boto3.client(
            's3',
            endpoint_url="http://192.168.80.155:32000",
            aws_access_key_id="admin",
            aws_secret_access_key="dlyticaD123",
            verify=False
        )
        
        current_date = datetime.now().strftime("%Y-%m-%d")
        minio_path = f"data/customer_segmentation_raw_{current_date}.csv"
        minio_client.upload_file(OUTPUT_FILE, minio_bucket, minio_path)
        
        return minio_path
        
    except Exception as e:
        print(f"Error in fetch_data_trino: {str(e)}")
        raise
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

@component(
    base_image='quay.io/datanature_dev/jupyternotebook:java_home14',
    packages_to_install=[
        'pyspark==3.5.0',
        'pandas==2.0.3',
        'numpy==1.24.4',  # Specific version compatible with pandas 2.0.3
        'boto3==1.28.57',
        'scikit-learn==1.3.0',
        'matplotlib==3.7.2',
        'pyarrow==12.0.1',
        'urllib3==2.0.4'
    ]
)
def feature_engineering_and_segmentation(
    file_path: str, 
    minio_bucket: str
) -> NamedTuple('Outputs', [
    ('segmentation_path', str),
    ('model_path', str),
    ('cluster_plot_path', str),
    ('cluster_interpretations', str)
]):
    # First ensure numpy is imported with the correct version
    import numpy as np
    np.__version__  # This helps catch version issues early
    
    # Then import other packages
    import boto3
    from botocore.config import Config
    import pandas as pd
    import matplotlib
    matplotlib.use('Agg')  # Set non-interactive backend
    import matplotlib.pyplot as plt
    from datetime import datetime
    import os
    import json
    import tempfile
    import time
    from collections import namedtuple
    
    # Now import pyspark components
    from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, when, mean, monotonically_increasing_id, stddev, count, abs
    from pyspark.sql.types import DoubleType, StringType
    from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA
    from pyspark.ml.clustering import BisectingKMeans
    from pyspark.ml.evaluation import ClusteringEvaluator
    from pyspark.sql.functions import udf

    def calculate_feature_distinctiveness(cluster_profiles):
        """Enhanced feature distinctiveness calculation"""
        feature_variance = cluster_profiles.std()
        distinctiveness = feature_variance / (feature_variance.max() + 1e-10)
        return distinctiveness

    def interpret_clusters(cluster_profiles):
        interpretations = {}
        distinctiveness = calculate_feature_distinctiveness(cluster_profiles)
        
        for cluster in cluster_profiles.index:
            features = cluster_profiles.columns.tolist()
            profile_components = []
            
            if 'purchases' in features:
                spend_level = cluster_profiles.loc[cluster, 'purchases']
                mean_spend = cluster_profiles['purchases'].mean()
                std_spend = cluster_profiles['purchases'].std()
                
                if spend_level > mean_spend + 1.5 * std_spend:
                    profile_components.append("High Spender")
                elif spend_level < mean_spend - 1.5 * std_spend:
                    profile_components.append("Low Spender")
                else:
                    profile_components.append("Moderate Spender")
            
            if 'cash_advance' in features and 'cash_advance_frequency' in features:
                cash_advance = cluster_profiles.loc[cluster, 'cash_advance']
                cash_freq = cluster_profiles.loc[cluster, 'cash_advance_frequency']
                mean_cash = cluster_profiles['cash_advance'].mean()
                mean_freq = cluster_profiles['cash_advance_frequency'].mean()
                
                if cash_advance > mean_cash * 2 and cash_freq > mean_freq * 2:
                    profile_components.append("Intensive Cash Advance User")
                elif cash_advance < mean_cash * 0.5 and cash_freq < mean_freq * 0.5:
                    profile_components.append("Rare Cash Advance User")
            
            if 'balance' in features and 'credit_limit' in features:
                balance_ratio = cluster_profiles.loc[cluster, 'balance'] / (cluster_profiles.loc[cluster, 'credit_limit'] + 1e-10)
                
                if balance_ratio > 0.8:
                    profile_components.append("Very High Credit Utilization")
                elif balance_ratio > 0.5:
                    profile_components.append("High Credit Utilization")
                elif balance_ratio < 0.2:
                    profile_components.append("Low Credit Utilization")
                else:
                    profile_components.append("Moderate Credit Utilization")
            
            if 'payments' in features and 'minimum_payments' in features:
                payment_ratio = cluster_profiles.loc[cluster, 'payments'] / (cluster_profiles.loc[cluster, 'minimum_payments'] + 1e-10)
                
                if payment_ratio > 3:
                    profile_components.append("Aggressive Overpayer")
                elif payment_ratio > 2:
                    profile_components.append("Consistent Overpayer")
                elif payment_ratio < 1.2:
                    profile_components.append("Minimum Payment User")
            
            if 'balance' in features and 'credit_limit' in features and 'cash_advance' in features:
                risk_score = (
                    cluster_profiles.loc[cluster, 'balance'] / (cluster_profiles.loc[cluster, 'credit_limit'] + 1e-10) * 0.4 +
                    cluster_profiles.loc[cluster, 'cash_advance'] / (cluster_profiles.loc[cluster, 'credit_limit'] + 1e-10) * 0.6
                )
                
                if risk_score > 0.7:
                    profile_components.append("Extremely High Financial Risk")
                elif risk_score > 0.5:
                    profile_components.append("High Financial Risk")
                elif risk_score < 0.2:
                    profile_components.append("Low Financial Risk")
                else:
                    profile_components.append("Moderate Financial Risk")
            
            if profile_components:
                interpretations[cluster] = "; ".join(profile_components)
            else:
                interpretations[cluster] = "Undefined Customer Segment"
        
        return interpretations

    def upload_to_minio(local_path, bucket, object_key, max_retries=5):
        """Uploads a file to MinIO with improved error handling and content-length issues fixed"""
        for attempt in range(max_retries):
            try:
                # Create a fresh client for each upload attempt
                client = boto3.client(
                    's3',
                    endpoint_url="http://192.168.80.155:32000",
                    aws_access_key_id="admin",
                    aws_secret_access_key="dlyticaD123",
                    verify=False,
                    config=Config(
                        connect_timeout=30,
                        read_timeout=60,
                        retries={'max_attempts': 3}
                    )
                )
                
                # Get the file size
                file_size = os.path.getsize(local_path)
                
                # For very small files or if multipart upload is causing issues, use a simpler method
                if file_size < 8 * 1024 * 1024:  # Less than 8MB
                    with open(local_path, 'rb') as file_data:
                        client.put_object(
                            Bucket=bucket,
                            Key=object_key,
                            Body=file_data,
                            ContentLength=file_size  # Explicitly set content length
                        )
                else:
                    # For larger files, we'll disable multipart upload and use a single put_object
                    # This may be slower for very large files but avoids the IncompleteBody error
                    with open(local_path, 'rb') as file_data:
                        data = file_data.read()
                        client.put_object(
                            Bucket=bucket,
                            Key=object_key,
                            Body=data,
                            ContentLength=len(data)  # Set exact content length
                        )
                    
                print(f"Successfully uploaded {local_path} to {bucket}/{object_key}")
                return f"{bucket}/{object_key}"
                
            except Exception as e:
                if attempt < max_retries - 1:
                    wait_time = 2 ** attempt  # Exponential backoff
                    print(f"Upload attempt {attempt+1} failed: {str(e)}. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    print(f"Upload failed after {max_retries} attempts: {str(e)}")
                    raise
                    
    # Initialize Spark with optimized settings
    spark = SparkSession.builder \
        .appName("AdvancedCustomerSegmentation") \
        .config("spark.driver.memory", "8g") \
        .config("spark.executor.memory", "8g") \
        .config("spark.sql.shuffle.partitions", "200") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "false") \
        .config("spark.executor.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true") \
        .config("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true") \
        .getOrCreate()

    # Initialize MinIO client with improved configuration
    boto_config = Config(
        connect_timeout=30,
        read_timeout=60,
        retries={'max_attempts': 3}
    )
    
    minio_client = boto3.client(
        's3',
        endpoint_url="http://192.168.80.155:32000",
        aws_access_key_id="admin",
        aws_secret_access_key="dlyticaD123",
        verify=False,
        config=boto_config
    )

    try:
        today = datetime.now().strftime('%Y-%m-%d')
        local_path = "/tmp/raw_data.csv"
        
        # Download with retry logic
        max_retries = 3
        for attempt in range(max_retries):
            try:
                minio_client.download_file(minio_bucket, file_path, local_path)
                break
            except Exception as e:
                if attempt == max_retries - 1:
                    raise
                time.sleep(5)

        # Read data with explicit schema
        df = spark.read \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .csv(local_path)

        if 'custid' in df.columns:
            df = df.withColumnRenamed("custid", "cif_id")

        # Feature engineering
        df = df.withColumn("balance_utilization_ratio", 
            when(col("credit_limit") != 0, col("balance") / col("credit_limit")).otherwise(0)
        )
        df = df.withColumn("cash_advance_intensity", 
            when(col("credit_limit") != 0, col("cash_advance") / col("credit_limit")).otherwise(0)
        )
        df = df.withColumn("payment_effort_ratio", 
            when(col("minimum_payments") != 0, col("payments") / col("minimum_payments")).otherwise(0)
        )
        df = df.withColumn("purchase_diversity", 
            abs(col("oneoff_purchases") - col("installments_purchases")) / 
            (abs(col("oneoff_purchases") + col("installments_purchases")) + 1e-10)
        )
        df = df.na.fill(0)

        numeric_cols = [
            'purchases', 'oneoff_purchases', 'installments_purchases', 
            'cash_advance', 'cash_advance_frequency', 
            'balance', 'credit_limit', 
            'payments', 'minimum_payments',
            'balance_utilization_ratio', 
            'cash_advance_intensity', 
            'payment_effort_ratio',
            'purchase_diversity'
        ]

        # Feature transformation
        assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")
        assembled = assembler.transform(df.drop("cif_id"))
        scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
        scaled_data = scaler.fit(assembled).transform(assembled)

        # Dimensionality reduction
        pca = PCA(k=5, inputCol="scaled_features", outputCol="pca_features")
        pca_data = pca.fit(scaled_data).transform(scaled_data)

        # Clustering optimization
        evaluator = ClusteringEvaluator(featuresCol='pca_features')
        k_range = range(6, 15)
        k_metrics = []

        for k in k_range:
            kmeans = BisectingKMeans(k=k, seed=42, featuresCol="pca_features")
            model = kmeans.fit(pca_data)
            predictions = model.transform(pca_data)
            
            cost = model.summary.trainingCost if hasattr(model.summary, 'trainingCost') else 0
            silhouette = evaluator.evaluate(predictions)
            
            centers = model.clusterCenters()
            separation = np.mean([np.min(np.linalg.norm(centers[i] - centers[j])) 
                                  for i in range(len(centers)) 
                                  for j in range(i+1, len(centers))])
            
            k_metrics.append({
                'k': k,
                'cost': cost,
                'silhouette': silhouette,
                'separation': separation
            })

        # Determine optimal k
        k_metrics_df = pd.DataFrame(k_metrics)
        k_metrics_df['normalized_cost'] = (k_metrics_df['cost'] - k_metrics_df['cost'].min()) / (k_metrics_df['cost'].max() - k_metrics_df['cost'].min())
        k_metrics_df['normalized_silhouette'] = (k_metrics_df['silhouette'] - k_metrics_df['silhouette'].min()) / (k_metrics_df['silhouette'].max() - k_metrics_df['silhouette'].min())
        k_metrics_df['normalized_separation'] = (k_metrics_df['separation'] - k_metrics_df['separation'].min()) / (k_metrics_df['separation'].max() - k_metrics_df['separation'].min())
        
        k_metrics_df['composite_score'] = (
            0.3 * (1 - k_metrics_df['normalized_cost']) +
            0.4 * k_metrics_df['normalized_silhouette'] +
            0.3 * k_metrics_df['normalized_separation']
        )
        
        optimal_k = k_metrics_df.loc[k_metrics_df['composite_score'].idxmax(), 'k']

        # Plotting
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15))
        ax1.plot(k_metrics_df['k'], k_metrics_df['cost'], marker='o', color='blue')
        ax1.set_xlabel('Number of Clusters (k)')
        ax1.set_ylabel('Cost', color='blue')
        ax1.grid(True)

        ax2.plot(k_metrics_df['k'], k_metrics_df['silhouette'], marker='x', color='green')
        ax2.set_xlabel('Number of Clusters (k)')
        ax2.set_ylabel('Silhouette Score', color='green')
        ax2.grid(True)

        ax3.plot(k_metrics_df['k'], k_metrics_df['separation'], marker='^', color='red')
        ax3.set_xlabel('Number of Clusters (k)')
        ax3.set_ylabel('Cluster Separation', color='red')
        ax3.grid(True)

        plt.tight_layout()
        cluster_plot_path = "/tmp/cluster_diagnostics.png"
        plt.savefig(cluster_plot_path)
        plt.close()
        
        # Final clustering
        kmeans = BisectingKMeans(k=optimal_k, seed=42, featuresCol="pca_features")
        model = kmeans.fit(pca_data)
        clustered_data = model.transform(pca_data)

        # Add cluster information
        df_with_id = df.withColumn("row_id", monotonically_increasing_id())
        clustered_data = clustered_data.withColumn("row_id", monotonically_increasing_id())
        clustered_data = clustered_data.join(df_with_id.select("row_id", "cif_id"), on="row_id").drop("row_id")

        # Generate cluster profiles
        summary = clustered_data.groupBy("prediction").agg(
            *[mean(col(c)).alias(c) for c in numeric_cols],
            *[stddev(col(c)).alias(f"{c}_std") for c in numeric_cols],
            count("cif_id").alias("cluster_size")
        )
        
        # Convert to Pandas safely
        cluster_data = summary.collect()
        cluster_profiles = pd.DataFrame([row.asDict() for row in cluster_data]).set_index("prediction")
        
        # Get interpretations
        interpretations = interpret_clusters(cluster_profiles)
        
        # Save outputs
        outputs = []
        output_paths = {
            'segmentation': "/tmp/segmentation_results.csv",
            'interpretations': "/tmp/cluster_interpretations.json",
            'model': "/tmp/segmentation_model",
            'plot': cluster_plot_path
        }
        
        # Save segmentation results - ensure file is closed properly
        results = clustered_data.select("cif_id", "prediction").collect()
        results_df = pd.DataFrame([row.asDict() for row in results])
        results_df.to_csv(output_paths['segmentation'], index=False)
        
        # Save interpretations - ensure file is closed properly
        with open(output_paths['interpretations'], 'w') as f:
            json.dump(interpretations, f)
            f.flush()
            os.fsync(f.fileno())
        
        # Save model
        model.write().overwrite().save(output_paths['model'])
        
        # Upload to MinIO with improved upload function
        minio_paths = {}
        
        # Upload single files first
        for name, local_path in output_paths.items():
            if name != 'model':  # Handle model directory separately
                minio_path = f"results/{name}_{today}{os.path.splitext(local_path)[1]}"
                try:
                    upload_to_minio(local_path, minio_bucket, minio_path)
                    minio_paths[name] = minio_path
                except Exception as e:
                    print(f"Error uploading {name}: {str(e)}")
                    raise
        
        # Handle model directory upload separately
        model_path = output_paths['model']
        minio_model_path = f"models/segmentation_model_{today}"
        model_files_uploaded = []
        
        try:
            for root, _, files in os.walk(model_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    relative_path = os.path.relpath(file_path, model_path)
                    s3_key = f"{minio_model_path}/{relative_path}"
                    upload_to_minio(file_path, minio_bucket, s3_key)
                    model_files_uploaded.append(s3_key)
            
            minio_paths['model'] = minio_model_path
        except Exception as e:
            print(f"Error uploading model files: {str(e)}")
            # Continue with other files even if model upload fails
            minio_paths['model'] = f"models/segmentation_model_{today}_partial"

        output = namedtuple('Outputs', [
            'segmentation_path',
            'model_path',
            'cluster_plot_path',
            'cluster_interpretations'
        ])
        
        return output(
            minio_paths.get('segmentation', ''),
            minio_paths.get('model', ''),
            minio_paths.get('plot', ''),
            minio_paths.get('interpretations', '')
        )
        
    except Exception as e:
        print(f"Error in feature_engineering_and_segmentation: {str(e)}")
        raise
    finally:
        # Make sure Spark is stopped properly
        if spark:
            spark.stop()


# Step 3: Save predictions to Trino
@component(
    base_image='quay.io/datanature_dev/jupyternotebook:java_home14',
    packages_to_install=['trino', 'pandas', 'boto3', 'urllib3']
)
def save_predictions_to_trino(
    segmentation_path: str,
    minio_bucket: str
) -> str:
    import trino
    from trino.auth import BasicAuthentication
    import pandas as pd
    import boto3
    import urllib3
    import pandas as pd
    from pyspark.sql import SparkSession
    from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
    from pyspark.ml import Pipeline
    from pyspark.ml.classification import LogisticRegression
    from pyspark.ml.evaluation import BinaryClassificationEvaluator
    import os
    import shutil
    from datetime import datetime
    from pyspark.sql.functions import lit, col, when
    
    # variables
    # variables
    # access_key_id = 'admin'
    # secret_access_key = 'dlyticaD123'
    # minio_endpoint = 'dn-minio-tenant-hl.dn-minio-tenant.svc.cluster.local:9000'  # Replace with your MinIO server endpoint
    # data_bucket = 'ai360fd-recommendation'
    # apps_bucket = ''
    # hive_metastore_uri = "thrift://dn-hive-metastore.dn-hive-metastore.svc.cluster.local:9083"
    # iceberg_warehouse_location = f"s3a://{data_bucket}/data/"
    # custom_catalog = "iceberg_catalog"
    # spark_eventlog_dir = f"s3a://{apps_bucket}/logs/spark/"
    # master_url="spark://dn-spark-master-svc.dn-spark.svc.cluster.local:7077"
    # spark_driver_host="notebook-five-spark-driver-headless.kubeflow-user-example-com.svc.cluster.local"
    # app_name="All Data Generation"
    # additional_packages="org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.2,org.apache.spark:spark-avro_2.12:3.5.0,com.crealytics:spark-excel_2.12:0.13.7,org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262,org.apache.hadoop:hadoop-common:3.3.4"

    
    access_key_id = 'admin'
    secret_access_key = 'dlyticaD123'
    minio_endpoint = 'dn-minio-tenant-hl.dn-minio-tenant.svc.cluster.local:9000'  # Replace with your MinIO server endpoint
    data_bucket = 'ai360fd-recommendation'
    # apps_bucket = 'dn-apps'
    hive_metastore_uri = "thrift://dn-hive-metastore.dn-hive-metastore.svc.cluster.local:9083"
    iceberg_warehouse_location = f"s3a://{data_bucket}/data/"
    custom_catalog = "iceberg_catalog"
    # spark_eventlog_dir = f"s3a://{apps_bucket}/logs/spark/"
    # master_url="spark://dn-spark-master-svc.dn-spark.svc.cluster.local:7077"
    # spark_driver_host="notebook-five-spark-driver-headless.kubeflow-user-example-com.svc.cluster.local"
    app_name="All Data Generation"
    additional_packages="org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.2,org.apache.spark:spark-avro_2.12:3.5.0,com.crealytics:spark-excel_2.12:0.13.7,org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262,org.apache.hadoop:hadoop-common:3.3.4"

    #spark configuration
    spark = SparkSession.builder \
        .appName(app_name) \
        .config("spark.driver.memory", "8g") \
        .config("spark.executor.memory", "8g") \
        .config("spark.executor.memoryOverhead", "2g") \
        .config("spark.driver.maxResultSize", "2g") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.hadoop.fs.s3a.endpoint", minio_endpoint) \
        .config("spark.hadoop.fs.s3a.access.key", access_key_id) \
        .config("spark.hadoop.fs.s3a.secret.key", secret_access_key) \
        .config("spark.hadoop.fs.s3a.path.style.access", "true") \
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
        .config("spark.hadoop.hive.metastore.uris", hive_metastore_uri) \
        .config("spark.sql.warehouse.dir", iceberg_warehouse_location) \
        .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog") \
        .config(f"spark.sql.catalog.{custom_catalog}", "org.apache.iceberg.spark.SparkCatalog") \
        .config("spark.jars.packages", additional_packages) \
        .config("spark.sql.repl.eagerEval.enabled", True) \
        .config("spark.sql.debug.maxToStringFields", 1000) \
        .enableHiveSupport() \
        .getOrCreate()
    
    # spark.sql(f"""
    # show tables in {custom_catalog}.gold
    # """).show(100,False)
    # print("train_model is called")
    # Initialize Spark session
    # spark = SparkSession.builder.appName("Feature Engineering").getOrCreate()
    
        
    # Disable SSL warnings
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

    try:
        # Download predictions from MinIO
        minio_client = boto3.client(
            's3',
            endpoint_url="http://192.168.80.155:32000",
            aws_access_key_id="admin",
            aws_secret_access_key="dlyticaD123",
            verify=False
        )
        
        local_path = "/tmp/predictions.csv"
        minio_client.download_file(minio_bucket, segmentation_path, local_path)
        predictions = pd.read_csv(local_path)
        
        # # Connect to Trino
        # conn = trino.dbapi.connect(
        #     host="192.168.80.155",
        #     port=30071,
        #     user="ctzn.bank",
        #     auth=BasicAuthentication("ctzn.bank", "ctzn.bank_123"),
        #     catalog="iceberg",
        #     schema="silver_crmuser",
        #     http_scheme="https",
        #     verify=False
        # )
        # cursor = conn.cursor()

        # # Create table if not exists
        # cursor.execute("""
        # CREATE TABLE IF NOT EXISTS iceberg.silver_crmuser.customer_segments (
        #     cif_id VARCHAR,
        #     cluster_id INTEGER,
        #     processed_date TIMESTAMP
        # ) WITH (
        #     partitioning = ARRAY['processed_date']
        # )
        # """)

        # Insert predictions in batches
        batch_size = 1000
        for i in range(0, len(predictions), batch_size):
            batch = predictions[i:i+batch_size]
            values = [(str(row['custid']), int(row['prediction'])) for _, row in batch.iterrows()]
            
            insert_sql = """
            INSERT INTO iceberg.silver_crmuser.customer_segments 
            (cif_id, cluster_id, processed_date) 
            VALUES (?, ?, CURRENT_TIMESTAMP)
            """
            
            cursor.executemany(insert_sql, values)
            conn.commit()

        return f"Successfully saved {len(predictions)} predictions to Trino"
        
    except Exception as e:
        print(f"Error in save_predictions_to_trino: {str(e)}")
        raise
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

# Pipeline definition
@pipeline(name="Customer Segmentation Pipeline")
def customer_segmentation_pipeline(
    minio_bucket: str = "ai360ctzn-customer-segmentation"
):
    # Step 1: Fetch data
    fetch_task = fetch_data_trino(minio_bucket=minio_bucket)
    
    # Step 2: Process data and create segments
    segmentation_task = feature_engineering_and_segmentation(
        file_path=fetch_task.output,
        minio_bucket=minio_bucket
    )
    
    # Step 3: Save results to Trino
    save_task = save_predictions_to_trino(
        segmentation_path=segmentation_task.outputs['segmentation_path'],
        minio_bucket=minio_bucket
    )

# Compile the pipeline
if __name__ == "__main__":
    from kfp import compiler
    compiler.Compiler().compile(
        customer_segmentation_pipeline,
        "customer_segmentation_pipeline.yaml"
    )
    print("Pipeline compiled successfully.")

Pipeline compiled successfully.


### FInal pipeline 

In [78]:
import kfp
from kfp.dsl import pipeline, component
from datetime import datetime
from typing import NamedTuple

# Step 1: Fetch data from Trino and save it to MinIO
@component(
    base_image='bitnami/spark:3.5', 
    packages_to_install=['trino', 'pandas', 'pyarrow', 's3fs', 'boto3', 'urllib3']
)
def fetch_data_trino(minio_bucket: str) -> str:
    import trino
    from trino.auth import BasicAuthentication
    import csv
    import boto3
    import os
    import urllib3
    from datetime import datetime
    
    # Disable SSL warnings
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

    # Trino Connection Details
    TRINO_HOST = "192.168.80.155"
    TRINO_PORT = "30071"
    TRINO_USER = "ctzn.bank"
    TRINO_PASSWORD = "ctzn.bank_123"
    TRINO_CATALOG = "iceberg"
    TRINO_SCHEMA = "silver_crmuser"
    TRINO_HTTP_SCHEME = "https"

    # Output file
    OUTPUT_FILE = "/tmp/ctzn_10_years_data.csv"

    # SQL Query
    SQL_QUERY = """
    WITH recent_customers AS (
        SELECT DISTINCT g.cif_id
        FROM gold.dim_gam AS g
        WHERE CAST(DATE_PARSE(SUBSTRING(g.acct_opn_date, 1, 19), '%Y-%m-%dT%H:%i:%s') AS DATE) >= CURRENT_DATE - INTERVAL '10' YEAR
    ),
    account_activity AS (
        SELECT 
            a.cif_id,
            SUM(COALESCE(a.total_credit_tran_vol, 0) - COALESCE(a.total_debit_tran_vol, 0)) AS balance,
            COUNT(DISTINCT a.nepali_month)/6.0 AS balance_frequency,
            SUM(COALESCE(a.total_debit_tran_vol, 0)) AS purchases,
            MAX(COALESCE(a.total_debit_tran_vol, 0)) AS oneoff_purchases,
            SUM(COALESCE(a.total_debit_tran_vol, 0)) - MAX(COALESCE(a.total_debit_tran_vol, 0)) AS installments_purchases,
            SUM(CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 AND COALESCE(a.total_debit_tran_vol, 0) = 0 
                    THEN COALESCE(a.total_credit_tran_vol, 0) ELSE 0 END) AS cash_advance,
            COUNT(DISTINCT a.foracid)/6.0 AS purchases_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_debit_tran_vol, 0) > 100000 THEN a.foracid END)/6.0 AS oneoff_purchases_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_debit_tran_vol, 0) BETWEEN 1 AND 100000 THEN a.foracid END)/6.0 AS purchases_installments_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 THEN a.foracid END)/6.0 AS cash_advance_frequency,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) > 0 THEN a.foracid END) AS cash_advance_trx,
            COUNT(DISTINCT a.foracid) AS purchases_trx,
            SUM(COALESCE(a.total_credit_tran_vol, 0)) AS payments,
            COUNT(DISTINCT CASE WHEN COALESCE(a.total_credit_tran_vol, 0) >= COALESCE(a.total_debit_tran_vol, 0) 
                                THEN a.nepali_month END)/6.0 AS prc_full_payment
        FROM gold.mv_fact_deposit_account_insights a
        JOIN recent_customers rc ON a.cif_id = rc.cif_id
        GROUP BY a.cif_id
    ),
    salary_stats AS (
        SELECT 
            APPROX_PERCENTILE(COALESCE(salary_per_month, 0), 0.5) AS median_salary,
            APPROX_PERCENTILE(COALESCE(salary_per_month, 0), 0.05) AS fifth_percentile_salary
        FROM gold.dim_customers
    ),
    customer_profile AS (
        SELECT 
            g.cif_id,
            DATE_DIFF('year', 
                     CAST(DATE_PARSE(SUBSTRING(MIN(g.acct_opn_date), 1, 19), '%Y-%m-%dT%H:%i:%s') AS DATE), 
                     CURRENT_DATE) AS tenure,
            (SELECT fifth_percentile_salary FROM salary_stats) AS minimum_payments
        FROM gold.dim_gam g
        LEFT JOIN gold.dim_customers c ON g.cif_id = c.cif_id
        GROUP BY g.cif_id
    )
    SELECT 
        aa.cif_id AS custid,
        aa.balance,
        aa.balance_frequency,
        aa.purchases,
        aa.oneoff_purchases,
        aa.installments_purchases,
        aa.cash_advance,
        aa.purchases_frequency,
        aa.oneoff_purchases_frequency,
        aa.purchases_installments_frequency,
        aa.cash_advance_frequency,
        aa.cash_advance_trx,
        aa.purchases_trx,
        (SELECT median_salary * 3 FROM salary_stats) AS credit_limit,
        aa.payments,
        cp.minimum_payments,
        aa.prc_full_payment,
        cp.tenure
    FROM account_activity aa
    JOIN customer_profile cp ON aa.cif_id = cp.cif_id
    ORDER BY aa.cif_id
    """

    try:
        # Connect to Trino
        conn = trino.dbapi.connect(
            host=TRINO_HOST,
            port=TRINO_PORT,
            user=TRINO_USER,
            auth=BasicAuthentication(TRINO_USER, TRINO_PASSWORD),
            catalog=TRINO_CATALOG,
            schema=TRINO_SCHEMA,
            http_scheme=TRINO_HTTP_SCHEME,
            request_timeout=600,
            verify=False
        )
        cursor = conn.cursor()
        
        # Execute query and fetch data
        cursor.execute(SQL_QUERY)
        columns = [desc[0] for desc in cursor.description]
        
        # Write to CSV in batches
        with open(OUTPUT_FILE, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(columns)
            
            while True:
                rows = cursor.fetchmany(1000)
                if not rows:
                    break
                writer.writerows(rows)
        
        # Upload to MinIO
        minio_client = boto3.client(
            's3',
            endpoint_url="http://192.168.80.155:32000",
            aws_access_key_id="admin",
            aws_secret_access_key="dlyticaD123",
            verify=False
        )
        
        current_date = datetime.now().strftime("%Y-%m-%d")
        minio_path = f"data/customer_segmentation_raw_{current_date}.csv"
        minio_client.upload_file(OUTPUT_FILE, minio_bucket, minio_path)
        
        return minio_path
        
    except Exception as e:
        print(f"Error in fetch_data_trino: {str(e)}")
        raise
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'conn' in locals():
            conn.close()

@component(
    base_image='quay.io/datanature_dev/jupyternotebook:java_home14',
    packages_to_install=[
        'pyspark==3.5.0',
        'pandas==2.0.3',
        'numpy==1.24.4',
        'boto3==1.28.57',
        'scikit-learn==1.3.0',
        'matplotlib==3.7.2',
        'pyarrow==12.0.1',
        'urllib3==2.0.4'
    ]
)
def feature_engineering_and_segmentation(
    file_path: str, 
    minio_bucket: str
) -> NamedTuple('Outputs', [
    ('segmentation_path', str),
    ('model_path', str),
    ('cluster_plot_path', str),
    ('cluster_interpretations', str),
    ('final_output_path', str)
]):
    import numpy as np
    import boto3
    import pandas as pd
    import matplotlib
    matplotlib.use('Agg')
    import matplotlib.pyplot as plt
    from datetime import datetime
    import os
    import json
    import time
    from collections import namedtuple

    from pyspark.sql import SparkSession
    from pyspark.sql.functions import col, when, mean, monotonically_increasing_id, stddev, count, abs, udf
    from pyspark.sql.types import StringType
    from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA
    from pyspark.ml.clustering import BisectingKMeans
    from pyspark.ml.evaluation import ClusteringEvaluator

    def interpret_clusters(cluster_profiles):
        interpretations = {}
        for cluster in cluster_profiles.index:
            profile_components = []
            spend = cluster_profiles.loc[cluster, 'purchases']
            spend_mean = cluster_profiles['purchases'].mean()
            spend_std = cluster_profiles['purchases'].std()
            if spend > spend_mean + 1.5 * spend_std:
                profile_components.append("High Spender")
            elif spend < spend_mean - 1.5 * spend_std:
                profile_components.append("Low Spender")
            else:
                profile_components.append("Moderate Spender")
            interpretations[cluster] = "; ".join(profile_components)
        return interpretations

    def upload_to_minio(local_path, bucket, object_key):
        client = boto3.client(
            's3',
            endpoint_url="http://192.168.80.155:32000",
            aws_access_key_id="admin",
            aws_secret_access_key="dlyticaD123",
            verify=False
        )
        client.upload_file(local_path, bucket, object_key)
        return f"{bucket}/{object_key}"

    spark = SparkSession.builder \
        .appName("CustomerSegmentation") \
        .config("spark.driver.memory", "8g") \
        .getOrCreate()

    try:
        today = datetime.now().strftime('%Y-%m-%d')
        local_path = "/tmp/raw_data.csv"
        boto_client = boto3.client(
            's3',
            endpoint_url="http://192.168.80.155:32000",
            aws_access_key_id="admin",
            aws_secret_access_key="dlyticaD123",
            verify=False
        )
        boto_client.download_file(minio_bucket, file_path, local_path)

        df = spark.read.option("header", True).option("inferSchema", True).csv(local_path)
        if 'custid' in df.columns:
            df = df.withColumnRenamed("custid", "cif_id")

        df = df.withColumn("balance_utilization_ratio", when(col("credit_limit") != 0, col("balance") / col("credit_limit")).otherwise(0))
        df = df.na.fill(0)

        numeric_cols = ['purchases', 'balance', 'credit_limit', 'balance_utilization_ratio']
        assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")
        scaled = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
        pca = PCA(k=3, inputCol="scaled_features", outputCol="pca_features")

        assembled = assembler.transform(df.drop("cif_id"))
        scaled_data = scaled.fit(assembled).transform(assembled)
        pca_data = pca.fit(scaled_data).transform(scaled_data)

        kmeans = BisectingKMeans(k=4, seed=42, featuresCol="pca_features")
        model = kmeans.fit(pca_data)
        clustered = model.transform(pca_data)

        df_id = df.withColumn("row_id", monotonically_increasing_id())
        clustered = clustered.withColumn("row_id", monotonically_increasing_id())
        clustered = clustered.join(df_id.select("row_id", "cif_id"), on="row_id").drop("row_id")

        summary = clustered.groupBy("prediction").agg(*[mean(col(c)).alias(c) for c in numeric_cols])
        cluster_profiles = pd.DataFrame([row.asDict() for row in summary.collect()]).set_index("prediction")
        interpretations = interpret_clusters(cluster_profiles)

        interpret_udf = udf(lambda x: interpretations.get(x, "Unknown"), StringType())
        clustered = clustered.withColumn("interpretation", interpret_udf(col("prediction")))

        final_df = clustered.select("cif_id", "prediction", "interpretation").toPandas()
        final_csv_path = f"/tmp/final_segmented_customers_{today}.csv"
        final_df.to_csv(final_csv_path, index=False)
        final_minio_path = f"results/final_segmented_customers_{today}.csv"
        upload_to_minio(final_csv_path, minio_bucket, final_minio_path)

        return namedtuple('Outputs', [
            'segmentation_path',
            'model_path',
            'cluster_plot_path',
            'cluster_interpretations',
            'final_output_path']) (
            '', '', '', '', final_minio_path
        )

    finally:
        spark.stop()


# # Step 3: Save predictions to Trino
# @component(
#     base_image='quay.io/datanature_dev/jupyternotebook:java_home14',
#     packages_to_install=['trino', 'pandas', 'boto3', 'urllib3']
# )
# def save_predictions_to_trino(
#     segmentation_path: str,
#     minio_bucket: str
# ) -> str:
#     import trino
#     from trino.auth import BasicAuthentication
#     import pandas as pd
#     import boto3
#     import urllib3
#     import pandas as pd
#     from pyspark.sql import SparkSession
#     from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
#     from pyspark.ml import Pipeline
#     from pyspark.ml.classification import LogisticRegression
#     from pyspark.ml.evaluation import BinaryClassificationEvaluator
#     import os
#     import shutil
#     from datetime import datetime
#     from pyspark.sql.functions import lit, col, when
    
#     # variables
#     # variables
#     # access_key_id = 'admin'
#     # secret_access_key = 'dlyticaD123'
#     # minio_endpoint = 'dn-minio-tenant-hl.dn-minio-tenant.svc.cluster.local:9000'  # Replace with your MinIO server endpoint
#     # data_bucket = 'ai360fd-recommendation'
#     # apps_bucket = ''
#     # hive_metastore_uri = "thrift://dn-hive-metastore.dn-hive-metastore.svc.cluster.local:9083"
#     # iceberg_warehouse_location = f"s3a://{data_bucket}/data/"
#     # custom_catalog = "iceberg_catalog"
#     # spark_eventlog_dir = f"s3a://{apps_bucket}/logs/spark/"
#     # master_url="spark://dn-spark-master-svc.dn-spark.svc.cluster.local:7077"
#     # spark_driver_host="notebook-five-spark-driver-headless.kubeflow-user-example-com.svc.cluster.local"
#     # app_name="All Data Generation"
#     # additional_packages="org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.2,org.apache.spark:spark-avro_2.12:3.5.0,com.crealytics:spark-excel_2.12:0.13.7,org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262,org.apache.hadoop:hadoop-common:3.3.4"

    
#     access_key_id = 'admin'
#     secret_access_key = 'dlyticaD123'
#     minio_endpoint = 'dn-minio-tenant-hl.dn-minio-tenant.svc.cluster.local:9000'  # Replace with your MinIO server endpoint
#     data_bucket = 'ai360fd-recommendation'
#     # apps_bucket = 'dn-apps'
#     hive_metastore_uri = "thrift://dn-hive-metastore.dn-hive-metastore.svc.cluster.local:9083"
#     iceberg_warehouse_location = f"s3a://{data_bucket}/data/"
#     custom_catalog = "iceberg_catalog"
#     # spark_eventlog_dir = f"s3a://{apps_bucket}/logs/spark/"
#     # master_url="spark://dn-spark-master-svc.dn-spark.svc.cluster.local:7077"
#     # spark_driver_host="notebook-five-spark-driver-headless.kubeflow-user-example-com.svc.cluster.local"
#     app_name="All Data Generation"
#     additional_packages="org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.2,org.apache.spark:spark-avro_2.12:3.5.0,com.crealytics:spark-excel_2.12:0.13.7,org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262,org.apache.hadoop:hadoop-common:3.3.4"

#     #spark configuration
#     spark = SparkSession.builder \
#         .appName(app_name) \
#         .config("spark.driver.memory", "8g") \
#         .config("spark.executor.memory", "8g") \
#         .config("spark.executor.memoryOverhead", "2g") \
#         .config("spark.driver.maxResultSize", "2g") \
#         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
#         .config("spark.hadoop.fs.s3a.endpoint", minio_endpoint) \
#         .config("spark.hadoop.fs.s3a.access.key", access_key_id) \
#         .config("spark.hadoop.fs.s3a.secret.key", secret_access_key) \
#         .config("spark.hadoop.fs.s3a.path.style.access", "true") \
#         .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
#         .config("spark.hadoop.hive.metastore.uris", hive_metastore_uri) \
#         .config("spark.sql.warehouse.dir", iceberg_warehouse_location) \
#         .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog") \
#         .config(f"spark.sql.catalog.{custom_catalog}", "org.apache.iceberg.spark.SparkCatalog") \
#         .config("spark.jars.packages", additional_packages) \
#         .config("spark.sql.repl.eagerEval.enabled", True) \
#         .config("spark.sql.debug.maxToStringFields", 1000) \
#         .enableHiveSupport() \
#         .getOrCreate()
    
#     # spark.sql(f"""
#     # show tables in {custom_catalog}.gold
#     # """).show(100,False)
#     # print("train_model is called")
#     # Initialize Spark session
#     # spark = SparkSession.builder.appName("Feature Engineering").getOrCreate()
    
        
#     # Disable SSL warnings
#     urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

#     try:
#         # Download predictions from MinIO
#         minio_client = boto3.client(
#             's3',
#             endpoint_url="http://192.168.80.155:32000",
#             aws_access_key_id="admin",
#             aws_secret_access_key="dlyticaD123",
#             verify=False
#         )
        
#         local_path = "/tmp/predictions.csv"
#         minio_client.download_file(minio_bucket, segmentation_path, local_path)
#         predictions = pd.read_csv(local_path)
        
#         # # Connect to Trino
#         # conn = trino.dbapi.connect(
#         #     host="192.168.80.155",
#         #     port=30071,
#         #     user="ctzn.bank",
#         #     auth=BasicAuthentication("ctzn.bank", "ctzn.bank_123"),
#         #     catalog="iceberg",
#         #     schema="silver_crmuser",
#         #     http_scheme="https",
#         #     verify=False
#         # )
#         # cursor = conn.cursor()

#         # # Create table if not exists
#         # cursor.execute("""
#         # CREATE TABLE IF NOT EXISTS iceberg.silver_crmuser.customer_segments (
#         #     cif_id VARCHAR,
#         #     cluster_id INTEGER,
#         #     processed_date TIMESTAMP
#         # ) WITH (
#         #     partitioning = ARRAY['processed_date']
#         # )
#         # """)

#         # Insert predictions in batches
#         batch_size = 1000
#         for i in range(0, len(predictions), batch_size):
#             batch = predictions[i:i+batch_size]
#             values = [(str(row['custid']), int(row['prediction'])) for _, row in batch.iterrows()]
            
#             insert_sql = """
#             INSERT INTO iceberg.silver_crmuser.customer_segments 
#             (cif_id, cluster_id, processed_date) 
#             VALUES (?, ?, CURRENT_TIMESTAMP)
#             """
            
#             cursor.executemany(insert_sql, values)
#             conn.commit()

#         return f"Successfully saved {len(predictions)} predictions to Trino"
        
#     except Exception as e:
#         print(f"Error in save_predictions_to_trino: {str(e)}")
#         raise
#     finally:
#         if 'cursor' in locals():
#             cursor.close()
#         if 'conn' in locals():
#             conn.close()

# Pipeline definition
@pipeline(name="Customer Segmentation Pipeline")
def customer_segmentation_pipeline(
    minio_bucket: str = "ai360ctzn-customer-segmentation"
):
    # Step 1: Fetch data
    fetch_task = fetch_data_trino(minio_bucket=minio_bucket)
    
    # Step 2: Process data and create segments
    segmentation_task = feature_engineering_and_segmentation(
        file_path=fetch_task.output,
        minio_bucket=minio_bucket
    )
    
    # # Step 3: Save results to Trino
    # save_task = save_predictions_to_trino(
    #     segmentation_path=segmentation_task.outputs['segmentation_path'],
    #     minio_bucket=minio_bucket
    # )

# Compile the pipeline
if __name__ == "__main__":
    from kfp import compiler
    compiler.Compiler().compile(
        customer_segmentation_pipeline,
        "customer_segmentation_pipeline.yaml"
    )
    print("Pipeline compiled successfully.")

Pipeline compiled successfully.


### Adding dex authentication

In [74]:
import re
from urllib.parse import urlsplit, urlencode

import kfp
import requests
import urllib3


class KFPClientManager:
    """
    A class that creates `kfp.Client` instances with Dex authentication.
    """

    def __init__(
        self,
        api_url: str,
        dex_username: str,
        dex_password: str,
        dex_auth_type: str = "local",
        skip_tls_verify: bool = False,
    ):
        """
        Initialize the KfpClient"

        :param api_url: the Kubeflow Pipelines API URL
        :param skip_tls_verify: if True, skip TLS verification
        :param dex_username: the Dex username
        :param dex_password: the Dex password
        :param dex_auth_type: the auth type to use if Dex has multiple enabled, one of: ['ldap', 'local']
        """
        self._api_url = api_url
        self._skip_tls_verify = skip_tls_verify
        self._dex_username = dex_username
        self._dex_password = dex_password
        self._dex_auth_type = dex_auth_type
        self._client = None

        # disable SSL verification, if requested
        if self._skip_tls_verify:
            urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

        # ensure `dex_default_auth_type` is valid
        if self._dex_auth_type not in ["ldap", "local"]:
            raise ValueError(
                f"Invalid `dex_auth_type` '{self._dex_auth_type}', must be one of: ['ldap', 'local']"
            )

    def _get_session_cookies(self) -> str:
        """
        Get the session cookies by authenticating against Dex
        :return: a string of session cookies in the form "key1=value1; key2=value2"
        """

        # use a persistent session (for cookies)
        s = requests.Session()

        # GET the api_url, which should redirect to Dex
        resp = s.get(
            self._api_url, allow_redirects=True, verify=not self._skip_tls_verify
        )
        if resp.status_code == 200:
            pass
        elif resp.status_code == 403:
            # if we get 403, we might be at the oauth2-proxy sign-in page
            # the default path to start the sign-in flow is `/oauth2/start?rd=<url>`
            url_obj = urlsplit(resp.url)
            url_obj = url_obj._replace(
                path="/oauth2/start", query=urlencode({"rd": url_obj.path})
            )
            resp = s.get(
                url_obj.geturl(), allow_redirects=True, verify=not self._skip_tls_verify
            )
        else:
            raise RuntimeError(
                f"HTTP status code '{resp.status_code}' for GET against: {self._api_url}"
            )

        # if we were NOT redirected, then the endpoint is unsecured
        if len(resp.history) == 0:
            # no cookies are needed
            return ""

        # if we are at `../auth` path, we need to select an auth type
        url_obj = urlsplit(resp.url)
        if re.search(r"/auth$", url_obj.path):
            url_obj = url_obj._replace(
                path=re.sub(r"/auth$", f"/auth/{self._dex_auth_type}", url_obj.path)
            )

        # if we are at `../auth/xxxx/login` path, then we are at the login page
        if re.search(r"/auth/.*/login$", url_obj.path):
            dex_login_url = url_obj.geturl()
        else:
            # otherwise, we need to follow a redirect to the login page
            resp = s.get(
                url_obj.geturl(), allow_redirects=True, verify=not self._skip_tls_verify
            )
            if resp.status_code != 200:
                raise RuntimeError(
                    f"HTTP status code '{resp.status_code}' for GET against: {url_obj.geturl()}"
                )
            dex_login_url = resp.url

        # attempt Dex login
        resp = s.post(
            dex_login_url,
            data={"login": self._dex_username, "password": self._dex_password},
            allow_redirects=True,
            verify=not self._skip_tls_verify,
        )
        if resp.status_code != 200:
            raise RuntimeError(
                f"HTTP status code '{resp.status_code}' for POST against: {dex_login_url}"
            )

        # if we were NOT redirected, then the login credentials were probably invalid
        if len(resp.history) == 0:
            raise RuntimeError(
                f"Login credentials are probably invalid - "
                f"No redirect after POST to: {dex_login_url}"
            )

        # if we are at `../approval` path, we need to approve the login
        url_obj = urlsplit(resp.url)
        if re.search(r"/approval$", url_obj.path):
            dex_approval_url = url_obj.geturl()

            # approve the login
            resp = s.post(
                dex_approval_url,
                data={"approval": "approve"},
                allow_redirects=True,
                verify=not self._skip_tls_verify,
            )
            if resp.status_code != 200:
                raise RuntimeError(
                    f"HTTP status code '{resp.status_code}' for POST against: {url_obj.geturl()}"
                )

        return "; ".join([f"{c.name}={c.value}" for c in s.cookies])

    def _create_kfp_client(self) -> kfp.Client:
        try:
            session_cookies = self._get_session_cookies()
        except Exception as ex:
            raise RuntimeError(f"Failed to get Dex session cookies") from ex

        # monkey patch the kfp.Client to support disabling SSL verification
        # kfp only added support in v2: https://github.com/kubeflow/pipelines/pull/7174
        original_load_config = kfp.Client._load_config

        def patched_load_config(client_self, *args, **kwargs):
            config = original_load_config(client_self, *args, **kwargs)
            config.verify_ssl = not self._skip_tls_verify
            return config

        patched_kfp_client = kfp.Client
        patched_kfp_client._load_config = patched_load_config

        return patched_kfp_client(
            host=self._api_url,
            cookies=session_cookies,
        )

    def create_kfp_client(self) -> kfp.Client:
        """Get a newly authenticated Kubeflow Pipelines client."""
        return self._create_kfp_client()

### Creating pipeline client and running pipeline dynamically

In [75]:
kfp_client_manager = KFPClientManager(
    api_url="http://192.168.80.155:31904/pipeline",
    skip_tls_verify=True,

    dex_username="user@dlytica.com",
    dex_password="dlytica@D123#",

    # can be 'ldap' or 'local' depending on your Dex configuration
    dex_auth_type="local",
)

# get a newly authenticated KFP client
# TIP: long-lived sessions might need to get a new client when their session expires
kfp_client = kfp_client_manager.create_kfp_client()

# # test the client by listing experiments
# experiments = kfp_client.list_experiments(namespace="kubeflow-user-example-com")
# print(experiments)


from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
pipeline_name = 'recommendation_pipeline'
pipelines = kfp_client.list_pipelines(page_size=100)
# experiments = kfp_client.list_experiments()
# print(experiments)
# print(pipelines)
for pipeline in pipelines.pipelines:
    print(pipeline.display_name)
    if pipeline.display_name == pipeline_name:
        print("pipeline matched")
        pipeline_id = pipeline.pipeline_id
        print(pipeline_id)
        # pipeline_id = '4758a6db-8057-463f-a185-a68f3e45d920'
        pipeline_versions = kfp_client.list_pipeline_versions(pipeline_id=pipeline_id, page_size=100)
        for version in pipeline_versions.pipeline_versions:
            print(f"Version ID: {version.pipeline_version_id}, Name: {version.display_name}")
        for version in pipeline_versions.pipeline_versions:
            version_id = version.pipeline_version_id
            try:
                kfp_client.delete_pipeline_version(pipeline_id = pipeline_id, pipeline_version_id=version_id)  
                print(f"Deleted version: {version_id}")
            except Exception as e:
                print(f"Failed to delete version {version_id}: {e}")
        kfp_client.delete_pipeline(pipeline.pipeline_id)
description = 'this is automl pipeline'
kfp_client.upload_pipeline('customer_segmentation_pipeline.yaml',pipeline_name = pipeline_name, description = description)

run = kfp_client.create_run_from_pipeline_func(
    customer_segmentation_pipeline,
    arguments={"minio_bucket": "ai360ctzn-customer-segmentation"}
)

/opt/conda/lib/python3.11/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


[Tutorial] Data passing in python components
[Tutorial] DSL - Control structures
Citizen_pipeline16
Customer_Segmentation_Pipeline
recommendation_pipeline
pipeline matched
600686ba-f0cd-4421-9d07-a3ae96f9e74f
Version ID: 872689fd-5ca7-44bb-bb11-e9b1eedb4aff, Name: recommendation_pipeline
Deleted version: 872689fd-5ca7-44bb-bb11-e9b1eedb4aff


Pipeline compiled successfully.


In [ ]:
SIR CLOSE GARNU PARCAH PLEASE SAVE UR WORK

SIR TODY OUR OFFICE TIME IS 1:30 
IM GETTING LATE 